
PROBLEM B5

Build and train a neural network model using the Daily Max Temperature.csv dataset.
Use MAE as the metrics of your neural network model.
We provided code for normalizing the data. Please do not change the code.
Do not use lambda layers in your model.
The dataset used in this problem is downloaded from https://github.com/jbrownlee/Datasets

Desired MAE < 0.2 on the normalized dataset.


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib

In [2]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [3]:
data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-max-temperatures.csv'
urllib.request.urlretrieve(data_url, 'daily-max-temperatures.csv')

time_step = []
temps = []

In [5]:
with open('daily-max-temperatures.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    step = 0
    for row in reader:
        # YOUR CODE HERE
        temps.append(float(row[1]))
        # YOUR CODE HERE
        time_step.append(step)
        step=step + 1

# YOUR CODE HERE
series= np.array(temps)

# Normalization Function. DO NOT CHANGE THIS CODE
min=np.min(series)
max=np.max(series)
series -= min
series /= max
time=np.array(time_step)

In [7]:
# DO NOT CHANGE THIS CODE
split_time=2500

# YOUR CODE HERE
time_train= time[:split_time]
x_train=series[:split_time]
time_valid=time[split_time:]
x_valid=series[split_time:]

# DO NOT CHANGE THIS CODE
window_size=64
batch_size=256
shuffle_buffer_size=1000

In [8]:
train_set=windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
print(train_set)
print(x_train.shape)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
(2500,)


In [14]:
model=tf.keras.models.Sequential([
    # YOUR CODE HERE.
    tf.keras.layers.Conv1D(filters = 32, kernel_size = 5, 
                           strides = 1, padding='causal',
                           activation='relu',
                           input_shape=[None, 1]),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1),
])

model.compile(loss=tf.keras.losses.Huber(), optimizer='adam', metrics=["mae"])

In [15]:
model.fit(train_set, epochs=10)

Epoch 1/10
10/10 [==============================] - 7s 54ms/step - loss: 0.0300 - mae: 0.2012
Epoch 2/10
10/10 [==============================] - 1s 54ms/step - loss: 0.0099 - mae: 0.1014
Epoch 3/10
10/10 [==============================] - 1s 53ms/step - loss: 0.0072 - mae: 0.0932
Epoch 4/10
10/10 [==============================] - 1s 59ms/step - loss: 0.0067 - mae: 0.0913
Epoch 5/10
10/10 [==============================] - 1s 54ms/step - loss: 0.0062 - mae: 0.0845
Epoch 6/10
10/10 [==============================] - 1s 53ms/step - loss: 0.0058 - mae: 0.0826
Epoch 7/10
10/10 [==============================] - 1s 52ms/step - loss: 0.0056 - mae: 0.0814
Epoch 8/10
10/10 [==============================] - 1s 54ms/step - loss: 0.0055 - mae: 0.0808
Epoch 9/10
10/10 [==============================] - 1s 52ms/step - loss: 0.0055 - mae: 0.0806
Epoch 10/10
10/10 [==============================] - 1s 54ms/step - loss: 0.0055 - mae: 0.0803


In [16]:
model.save("model_B5.h5")